In [16]:
# DATA STANDARDIZATION

# import libraries
import os
import shutil
from collections import defaultdict 

# Directories
input_folder = r'C:\Users\Public\PartIIB project 2023_2024\Image collection without reaction\00AgNO3_mole_fraction\allGeometryExcludeHelical2STD'
output_folder_train = r'C:\Users\Public\PartIIB project 2023_2024\Image collection without reaction\00AgNO3_mole_fraction\dataSelectorAllGeometryExcludeHelicalTwoLowerLimitSTD\train'
output_folder_test_val = r'C:\Users\Public\PartIIB project 2023_2024\Image collection without reaction\00AgNO3_mole_fraction\dataSelectorAllGeometryExcludeHelicalTwoLowerLimitSTD\test_val'
output_folder_test_val_disc = r'C:\Users\Public\PartIIB project 2023_2024\Image collection without reaction\00AgNO3_mole_fraction\dataSelectorAllGeometryExcludeHelicalTwoLowerLimitSTD\test_val_disc'

# Self-define parameters

# number of image in input
input_num = 4
#low limit size 
lower_limit = 0
#upper limit size
upper_limit = 4
# the size range of each bin
bin_width = 0.5
# number of size bins
num_bins = int((upper_limit-lower_limit)/bin_width)

# disrgardFactor x x_avg (average number of contours per bin) = the upper limit such that any bin with lower than this number is discarded
disregardFactor = 0.5

# propotion of data that will be used for traning given disrgardFactor x x_avg < x < x_avg x upperFactor / train_proportion
trainProportion = 0.8

# x_avg x upperFactor / train_proportion = below this number take 80% to train and above this number take x_avg x upperFactor to train
upperFactor = 2




def obtainSizeRange(targetDir, input_num, num_bins, lower_limit=7.0, upper_limit=14.0):
    lstDir = os.listdir(targetDir)    
    numLst = [i/2 + lower_limit for i in range(num_bins)]
    print(f'numLst: {numLst}')
    sizeDict = {}
    for j in numLst:
      sizeDict[j] = 0
    for i in lstDir:
        t_number = int(i.split("t-")[1].split("_")[0])
        lowerT = 200 - (200/input_num) + 1
        if lowerT <= t_number <= 200:
            size = float(i[-17:-5])
            if lower_limit < size < upper_limit:
                sizeRange = int(size*2)/2
                sizeDict[sizeRange] += 1

    # print(sizeDict)
    print(dict(sorted(sizeDict.items())))
    print(dict(sorted(sizeDict.items(), key=lambda item: item[1])))
    return dict(sorted(sizeDict.items()))

sizeDict = obtainSizeRange(targetDir = input_folder, input_num= input_num, num_bins= num_bins, lower_limit= lower_limit, upper_limit= upper_limit)
totalItems = sum(sizeDict.values())
print(totalItems)

def obtainDesiredDict(sizeDict, disregardFactor, trainProportion, upperFactor, lower_limit=7.0):
    # get avg
    print(len(sizeDict))
    numLst = [i/2 + lower_limit for i in range(len(sizeDict)+1)]

    counter = 0
    for j in numLst:
        try:
            counter += sizeDict[j]
        except KeyError:
            pass    
    x_avg = counter/(len(numLst))
    
    print(f'x_avg = {x_avg}')
    
    desiredDict = {}
    x_lower = x_avg*disregardFactor
    x_upper = (x_avg*upperFactor)/trainProportion
    
    for j in numLst:
        try:
            # x = number of contours per bin
            x = sizeDict[j]

            if x < x_lower:
                desiredDict[j] = 0
            
            elif x_lower < x < x_upper:
                desiredDict[j] = int(trainProportion*x)   
            elif x > x_upper:
                desiredDict[j] = int(x_avg*upperFactor)
                
        except KeyError:
            pass
    return desiredDict

desiredDict = obtainDesiredDict(sizeDict, disregardFactor= disregardFactor, trainProportion= trainProportion, upperFactor= upperFactor, lower_limit= lower_limit)
# print(desiredDict)

def data_standardisation(input_folder, output_folder_train, output_folder_test_val, desiredDict, input_num, bin_width, output_folder_test_val_disc, lower_limit=7.0):

   last_time_section = ((input_num-1)/(input_num))*200
   os.makedirs(output_folder_train, exist_ok=True)
   os.makedirs(output_folder_test_val, exist_ok=True)
   os.makedirs(output_folder_test_val_disc, exist_ok=True)
   
   
   # create the counter dictionary
   numLst = [i/2 + lower_limit for i in range(len(desiredDict))]
   counterDict = {}
   for j in numLst:
      counterDict[j] = 0
      

   for filename in os.listdir(input_folder):
      try:
         size = float(filename[-17:-5])
         t_number = int(filename.split("t-")[1].split("_")[0])
         if t_number > last_time_section:
            for j in numLst:
               # print(j)
               # print(size)
               if j < size < j+bin_width:
                  # print(counterDict[j])
                  # print(desiredDict[j])
                  if counterDict[j] < desiredDict[j]:
                     input_filepath = os.path.join(input_folder, filename)
                     output_filepath = os.path.join(output_folder_train, filename)
                     shutil.copy2(input_filepath, output_filepath)
                     counterDict[j] += 1
                     # print(counterDict[j])
                     for j in range(input_num-1):
                           fig_number = filename.split("_")[1]
                           t_number_new = str(int(t_number-(j+1)*(200/input_num)))
                           filename_new = [item for item in os.listdir(input_folder) if item.startswith(f"Fig_{fig_number}__t-{t_number_new}_")][0]
                           input_filepath = os.path.join(input_folder, filename_new)
                           output_filepath = os.path.join(output_folder_train, filename_new)
                           shutil.copy2(input_filepath, output_filepath)
      except (ValueError, IndexError):
         pass
   for filename2 in os.listdir(input_folder):
      if filename2 not in os.listdir(output_folder_train):
         try:
            size = float(filename2[-17:-5])
            t_number = int(filename2.split("t-")[1].split("_")[0])
            if t_number > last_time_section:
               for j in numLst:
                  if j < size < j+bin_width:
                     if desiredDict[j] < 1:
                        input_filepath = os.path.join(input_folder, filename2)
                        output_filepath = os.path.join(output_folder_test_val_disc, filename2)
                        shutil.copy2(input_filepath, output_filepath)
                        for j in range(input_num-1):
                           fig_number = filename2.split("_")[1]
                           t_number_new = str(int(t_number-(j+1)*(200/input_num)))
                           filename_new = [item for item in os.listdir(input_folder) if item.startswith(f"Fig_{fig_number}__t-{t_number_new}_")][0]
                           input_filepath = os.path.join(input_folder, filename_new)
                           output_filepath = os.path.join(output_folder_test_val_disc, filename_new)
                           shutil.copy2(input_filepath, output_filepath)
                     else:
                        input_filepath = os.path.join(input_folder, filename2)
                        output_filepath = os.path.join(output_folder_test_val, filename2)
                        shutil.copy2(input_filepath, output_filepath)
                        for j in range(input_num-1):
                           fig_number = filename2.split("_")[1]
                           t_number_new = str(int(t_number-(j+1)*(200/input_num)))
                           filename_new = [item for item in os.listdir(input_folder) if item.startswith(f"Fig_{fig_number}__t-{t_number_new}_")][0]
                           input_filepath = os.path.join(input_folder, filename_new)
                           output_filepath = os.path.join(output_folder_test_val, filename_new)
                           shutil.copy2(input_filepath, output_filepath)
         except:
            pass

   # print(counterDict)
   return counterDict
      
               

counterDict = data_standardisation(input_folder=input_folder, output_folder_train=output_folder_train, output_folder_test_val=output_folder_test_val, desiredDict=desiredDict, input_num=input_num, bin_width= bin_width, lower_limit=lower_limit, output_folder_test_val_disc=output_folder_test_val_disc)
counterDict

numLst: [0.0, 0.5, 1.0, 1.5, 2.0, 2.5, 3.0, 3.5]
{0.0: 653, 0.5: 357, 1.0: 257, 1.5: 207, 2.0: 180, 2.5: 145, 3.0: 81, 3.5: 97}
{3.0: 81, 3.5: 97, 2.5: 145, 2.0: 180, 1.5: 207, 1.0: 257, 0.5: 357, 0.0: 653}
1977
8
x_avg = 219.66666666666666


{0.0: 439, 0.5: 285, 1.0: 205, 1.5: 165, 2.0: 144, 2.5: 116, 3.0: 0, 3.5: 0}

In [15]:
import matplotlib.pyplot as plt
import os
import shutil
from collections import defaultdict 

# Directories
input_folder = r'C:\Users\Public\PartIIB project 2023_2024\Image collection without reaction\00AgNO3_mole_fraction\allGeometryExcludeHelical2STD'
# Self-define parameters

# number of image in input
input_num = 4
#low limit size 
lower_limit = 0
#upper limit size
upper_limit = 4
# the size range of each bin
bin_width = 0.5
# number of size bins
num_bins = int((upper_limit-lower_limit)/bin_width)

# disrgardFactor x x_avg (average number of contours per bin) = the upper limit such that any bin with lower than this number is discarded
disregardFactor = 0.5

# propotion of data that will be used for traning given disrgardFactor x x_avg < x < x_avg x upperFactor / train_proportion
trainProportion = 0.8

# x_avg x upperFactor / train_proportion = below this number take 80% to train and above this number take x_avg x upperFactor to train
upperFactor = 4




def obtainSizeRange(targetDir, input_num, num_bins, lower_limit=7.0, upper_limit=14.0):
    lstDir = os.listdir(targetDir)    
    numLst = [i/2 + lower_limit for i in range(num_bins)]
    print(f'numLst: {numLst}')
    sizeDict = {}
    for j in numLst:
      sizeDict[j] = 0
    for i in lstDir:
        t_number = int(i.split("t-")[1].split("_")[0])
        lowerT = 200 - (200/input_num) + 1
        if lowerT <= t_number <= 200:
            size = float(i[-17:-5])
            if lower_limit < size < upper_limit:
                sizeRange = int(size*2)/2
                sizeDict[sizeRange] += 1

    # print(sizeDict)
    print(dict(sorted(sizeDict.items())))
    print(dict(sorted(sizeDict.items(), key=lambda item: item[1])))
    return dict(sorted(sizeDict.items()))

sizeDict = obtainSizeRange(targetDir = input_folder, input_num= input_num, num_bins= num_bins, lower_limit= lower_limit, upper_limit= upper_limit)
totalItems = sum(sizeDict.values())
print(totalItems)

def obtainDesiredDict(sizeDict, disregardFactor, trainProportion, upperFactor, lower_limit=7.0):
    # get avg
    print(len(sizeDict))
    numLst = [i/2 + lower_limit for i in range(len(sizeDict)+1)]

    counter = 0
    for j in numLst:
        try:
            counter += sizeDict[j]
        except KeyError:
            pass    
    x_avg = counter/(len(numLst))
    
    print(f'x_avg = {x_avg}')
    
    desiredDict = {}
    x_lower = x_avg*disregardFactor
    x_upper = (x_avg*upperFactor)/trainProportion
    
    for j in numLst:
        try:
            # x = number of contours per bin
            x = sizeDict[j]

            if x < x_lower:
                desiredDict[j] = 0
            
            elif x_lower < x < x_upper:
                desiredDict[j] = int(trainProportion*x)   
            elif x > x_upper:
                desiredDict[j] = int(x_avg*upperFactor)
                
        except KeyError:
            pass
    return desiredDict

desiredDict = obtainDesiredDict(sizeDict, disregardFactor= disregardFactor, trainProportion= trainProportion, upperFactor= upperFactor, lower_limit= lower_limit)

desiredDict = obtainDesiredDict(sizeDict, disregardFactor= disregardFactor, trainProportion= trainProportion, upperFactor= upperFactor, lower_limit= lower_limit)
totalDesiredDict = sum(desiredDict.values())
print(f'total desiredDict items: {totalDesiredDict}')
plt.bar(list(desiredDict.keys()), desiredDict.values(), -0.5, color='g')
plt.show

numLst: [0.0, 0.5, 1.0, 1.5, 2.0, 2.5, 3.0, 3.5]


KeyError: 9.0